In [1]:
import tensorflow as tf
import os
import random
import numpy as np

def seed_everything(seed):
  random.seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)
  np.random.seed(seed)
  tf.random.set_seed(seed)

SEED = 22
seed_everything(seed=SEED)

In [2]:
from functools import reduce
from itertools import accumulate

landmark_lens = (
    (33, 4),
    (468, 3),
    (21, 3),
    (21, 3)
)
landmark_locs = list(accumulate(landmark_lens, lambda a, b: a + b[0]*b[1], initial=0))
landmarks_len = reduce(lambda r, loc: r + loc[0] * loc[1], landmark_lens, 0)
print(landmark_locs, landmarks_len)

[0, 132, 1536, 1599, 1662] 1662


In [3]:
# data_folder = 'tracks_binary_manual'
data_folder = 'tracks_binary'
labels = [label for label in os.listdir(data_folder) if os.path.isdir(f'{data_folder}/{label}')]
NUM_CLASSES = len(labels)

labels_tensor = tf.constant(labels)
ids_tensor = tf.constant(range(len(labels)))

ids_from_labels = tf.lookup.StaticHashTable(
    tf.lookup.KeyValueTensorInitializer(
        labels_tensor,
        ids_tensor
    ),
    default_value=-1
)

labels_from_ids = tf.lookup.StaticHashTable(
    tf.lookup.KeyValueTensorInitializer(
        ids_tensor,
        labels_tensor
    ),
    default_value=""
)

def to_categorical(label):
    return tf.one_hot(
        ids_from_labels.lookup(label),
        depth=NUM_CLASSES
    )

In [4]:
def process_binary(file_path):
    label = tf.strings.split(file_path, os.sep)[-2]

    raw = tf.io.read_file(file_path)
    data = tf.io.decode_raw(raw, tf.float32)
    data = tf.reshape(data, [-1, landmarks_len])

    pose = tf.reshape(data[:, 0:132], [-1, 33, 4])
    # lh = tf.reshape(data[:, 132:195], [-1, 21, 3])
    # rh = tf.reshape(data[:, 195:258], [-1, 21, 3])
    
    face = tf.reshape(data[:, 132:1536], [-1, 468, 3])
    lh = tf.reshape(data[:, 1536:1599], [-1, 21, 3])
    rh = tf.reshape(data[:, 1599:1662], [-1, 21, 3])
    
    # without z
    new_pose = tf.concat((pose[:, :, :2], pose[:, :, -1:]), axis=2)
    new_lh = lh[:, :, :2]
    new_rh = rh[:, :, :2]

    return (new_pose, new_lh, new_rh), to_categorical(label)
    # return (pose, face, lh, rh), to_categorical(label)

In [5]:
from tensorflow import reduce_max, reduce_min

FRAMES = 64

def flatten(x):
    pose = tf.reshape(x[0], shape=[-1, 33*3])
    # face = tf.reshape(x[1], shape=[-1, 1404])
    lh = tf.reshape(x[1], shape=[-1, 21*2])
    rh = tf.reshape(x[2], shape=[-1, 21*2])
    return tf.concat([pose, lh, rh], axis=1)


def random_window(x):
    def pad(x):
        missing = FRAMES - size
        start_pad = tf.math.ceil(missing / 2)
        end_pad = tf.math.floor(missing / 2)
        return tf.concat([
            tf.tile([x[0]], [start_pad, 1]),
            x,
            tf.tile([x[-1]], [end_pad, 1])
        ], axis=0)

    def random_slice(x):
        i = tf.random.uniform(shape=(), maxval=size+1-FRAMES, dtype=tf.int32)
        return x[i: i+FRAMES]

    size = tf.shape(x)[0]
    print(size)
    return tf.cond(
        size < FRAMES,
        lambda: pad(x),
        lambda: random_slice(x)
    )
    
def calc_bounding(pose, lh, rh):
    max_x = reduce_max(tf.stack([reduce_max(pose[:, :, :1]), reduce_max(lh[:, :, :1]), reduce_max(rh[:, :, :1])], axis=0))
    min_x = reduce_min(tf.stack([reduce_min(pose[:, :, :1]), reduce_min(lh[:, :, :1]), reduce_min(rh[:, :, :1])], axis=0))
    
    max_y = reduce_max(tf.stack([reduce_max(pose[:, :, 1:2]), reduce_max(lh[:, :, 1:2]), reduce_max(rh[:, :, 1:2])], axis=0))
    min_y = reduce_min(tf.stack([reduce_min(pose[:, :, 1:2]), reduce_min(lh[:, :, 1:2]), reduce_min(rh[:, :, 1:2])], axis=0))

    window = tf.cast((max_x - min_x, max_y - min_y), dtype=tf.float32)
    mid = ((max_x + min_x)/2, (max_y + min_y)/2)
    return (window, mid)

def scale(x, factor):
    pose, lh, rh = x[0], x[1], x[2]
    window, mid = calc_bounding(pose, lh, rh)
    scale = factor * window
    pose_shape, lh_shape, rh_shape = tf.shape(pose), tf.shape(lh), tf.shape(rh)
    
    pose_center = tf.tile([[[mid[0], mid[1], 0]]], [pose_shape[0], pose_shape[1], 1])
    lh_center = tf.tile([[[mid[0], mid[1]]]], [lh_shape[0], lh_shape[1], 1])
    rh_center = tf.tile([[[mid[0], mid[1]]]], [rh_shape[0], rh_shape[1], 1])
    
    pose_scale = tf.tile([[[scale[0], scale[1], 1]]], [pose_shape[0], pose_shape[1], 1])
    lh_scale = tf.tile([[[scale[0], scale[1]]]], [lh_shape[0], lh_shape[1], 1])
    rh_scale = tf.tile([[[scale[0], scale[1]]]], [rh_shape[0], rh_shape[1], 1])
    
    scaled_pose = pose_center + (pose - pose_center) * pose_scale
    scaled_lh = lh_center + (lh - lh_center) * lh_scale
    scaled_rh = rh_center + (rh - rh_center) * rh_scale

    return (scaled_pose, scaled_lh, scaled_rh)


def random_translation(x):
    pose, lh, rh = x[0], x[1], x[2]
    magnitude = tf.random.uniform(shape=[2], minval=-0.25, maxval=0.25)
    pose_shape, lh_shape, rh_shape = tf.shape(pose), tf.shape(lh), tf.shape(rh)
    
    pose_trans = tf.tile([[[magnitude[0], magnitude[1], 0]]], [pose_shape[0], pose_shape[1], 1])
    lh_trans = tf.tile([[[magnitude[0], magnitude[1]]]], [lh_shape[0], lh_shape[1], 1])
    rh_trans = tf.tile([[[magnitude[0], magnitude[1]]]], [rh_shape[0], rh_shape[1], 1])

    return (pose+pose_trans, lh+lh_trans, rh+rh_trans)
    
def flip(x):
    pose, lh, rh = x[0], x[1], x[2]
    pose_shape, lh_shape, rh_shape = tf.shape(pose), tf.shape(lh), tf.shape(rh)
    
    pose_neg = tf.tile([[[-1.0, 1, 1]]], [pose_shape[0], pose_shape[1], 1])
    lh_neg = tf.tile([[[-1.0, 1]]], [lh_shape[0], lh_shape[1], 1])
    rh_neg = tf.tile([[[-1.0, 1]]], [rh_shape[0], rh_shape[1], 1])
    
    pose_trans = tf.tile([[[1.0, 0, 0]]], [pose_shape[0], pose_shape[1], 1])
    lh_trans = tf.tile([[[1.0, 0]]], [lh_shape[0], lh_shape[1], 1])
    rh_trans = tf.tile([[[1.0, 0]]], [rh_shape[0], rh_shape[1], 1])

    flipped_pose = pose_trans + pose * pose_neg
    flipped_lh = lh_trans + lh * lh_neg
    flipped_rh = rh_trans + rh * rh_neg
    
    return (flipped_pose, flipped_lh, flipped_rh)
    
def prepare(ds, shuffle=False, augment=False):
    if augment:
        ds = ds.map(lambda x, y: (random_translation(x), y), num_parallel_calls=tf.data.AUTOTUNE)
        ds = ds.map(lambda x, y: (scale(x, 0.1), y), num_parallel_calls=tf.data.AUTOTUNE)
        ds = ds.map(lambda x, y: (flip(x), y), num_parallel_calls=tf.data.AUTOTUNE)
               
    ds = ds.map(lambda x, y: (flatten(x), y), num_parallel_calls=tf.data.AUTOTUNE)

    ds = ds.map(lambda x, y: (random_window(x), y), num_parallel_calls=tf.data.AUTOTUNE)

    if shuffle:
        ds = ds.shuffle(1000, seed=SEED)
        
    ds = ds.batch(32)

    return ds.prefetch(buffer_size=tf.data.AUTOTUNE)

In [6]:
def get_ds_split(ds, ds_size, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=1000):
  assert (train_split + test_split + val_split) == 1
  
  if shuffle:
    ds = ds.shuffle(shuffle_size, seed=SEED)
  
  train_size = int(train_split * ds_size)
  val_size = int(val_split * ds_size)
  
  train_ds = ds.take(train_size)
  val_ds = ds.skip(train_size).take(val_size)
  test_ds = ds.skip(train_size).skip(val_size)
  
  return train_ds, val_ds, test_ds

In [18]:
isAugment = True

ds = tf.data.Dataset.list_files(f'{data_folder}/*/*')
ds = ds.map(process_binary)
print(ds)
train_ds, val_ds, test_ds = get_ds_split(ds, len(ds))
print(train_ds)

train_ds = prepare(train_ds, augment=isAugment)
val_ds = prepare(val_ds)
test_ds = prepare(test_ds)

<MapDataset shapes: (((None, 33, 3), (None, 21, 2), (None, 21, 2)), (10,)), types: ((tf.float32, tf.float32, tf.float32), tf.float32)>
<TakeDataset shapes: (((None, 33, 3), (None, 21, 2), (None, 21, 2)), (10,)), types: ((tf.float32, tf.float32, tf.float32), tf.float32)>
Tensor("strided_slice:0", shape=(), dtype=int32)
Tensor("strided_slice:0", shape=(), dtype=int32)
Tensor("strided_slice:0", shape=(), dtype=int32)


In [19]:
train_ds

<PrefetchDataset shapes: ((None, None, 183), (None, 10)), types: (tf.float32, tf.float32)>

In [20]:
val_ds

<PrefetchDataset shapes: ((None, None, 183), (None, 10)), types: (tf.float32, tf.float32)>

In [21]:
print(len(train_ds), len(val_ds), len(test_ds))

8 1 2


In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ReduceLROnPlateau
from tensorflow import keras
from wandb.keras import WandbCallback
import wandb

In [13]:
wandb.init(
  project="bi-LSTM",
  entity="richardsonqiu", 
  config={
    "input_shape": (FRAMES, 183),
    
    "lstm_1": 120,
    "layer_1": 120,
    "act_1": "relu",
    
    "lstm_2": 130,
    # "dropout_2": 0.1,
    "layer_2": 130,
    "act_2": "relu",
        
    "lstm_3": 200,
    "dropout_3": 0.2,
    
    "last_layer": NUM_CLASSES,
    "last_act": "softmax",
    
    "optimizer": "adam",
    "init_lr": 0.01,
    "loss": "categorical_crossentropy",
    "metric": "accuracy",
    "epoch": 1000,
    "batch_size": 32,
    "data": "default",
    "landmarks": "pose, lh, rh",
    "landmarks_metadata": "without z",
    "augment": isAugment  
    })
config = wandb.config

model = Sequential()
model.add(Bidirectional(LSTM(config.lstm_1, return_sequences=True), input_shape=(config.input_shape[0], config.input_shape[1])))
model.add(Dense(config.layer_1, activation=config.act_1))
model.add(Bidirectional(LSTM(config.lstm_2, return_sequences=True)))
model.add(Dense(config.layer_2, activation=config.act_2))
model.add(Bidirectional(LSTM(config.lstm_3, return_sequences=False, dropout=config.dropout_3)))
model.add(Dense(config.last_layer, activation=config.last_act))

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: richardsonqiu (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.9 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [14]:
if config.optimizer == "adam":
  opt = keras.optimizers.Adam(learning_rate=config.init_lr)
elif config.optimizer == "sgd":
  opt = keras.optimizers.SGD(learning_rate=config.init_lr, nesterov=True)
  
model.compile(optimizer=opt, loss=config.loss, metrics=[config.metric])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 64, 240)          291840    
 l)                                                              
                                                                 
 dense (Dense)               (None, 64, 120)           28920     
                                                                 
 bidirectional_1 (Bidirectio  (None, 64, 260)          261040    
 nal)                                                            
                                                                 
 dense_1 (Dense)             (None, 64, 130)           33930     
                                                                 
 bidirectional_2 (Bidirectio  (None, 400)              529600    
 nal)                                                            
                                                        

In [16]:
es_callback = EarlyStopping(monitor='val_loss', patience=20)
lr_callback = ReduceLROnPlateau(monitor='val_loss', patience=20, factor=0.5, min_lr=1e-6)
wandb_callback = WandbCallback(log_evaluation=True)

In [17]:
# history = model.fit(train_ds, validation_data=val_ds, epochs=config.epoch, callbacks=[lr_callback, wandb_callback])
history = model.fit(train_ds, validation_data=val_ds, epochs=config.epoch, callbacks=[lr_callback, wandb_callback])

wandb: WARNING WandbCallback is unable to log validation data. When using a generator for validation_data, you must pass validation_steps


Epoch 1/1000
8/8 [==============================] - 21s 781ms/step - loss: 2.6265 - accuracy: 0.1294 - val_loss: 2.2399 - val_accuracy: 0.1290 - lr: 0.0100
Epoch 2/1000
8/8 [==============================] - 2s 236ms/step - loss: 2.2872 - accuracy: 0.1451 - val_loss: 2.3108 - val_accuracy: 0.1290 - lr: 0.0100
Epoch 3/1000
8/8 [==============================] - 2s 243ms/step - loss: 2.2808 - accuracy: 0.1255 - val_loss: 2.2960 - val_accuracy: 0.1935 - lr: 0.0100
Epoch 4/1000
8/8 [==============================] - 2s 236ms/step - loss: 2.2436 - accuracy: 0.1255 - val_loss: 2.2065 - val_accuracy: 0.1613 - lr: 0.0100
Epoch 5/1000
8/8 [==============================] - 2s 237ms/step - loss: 2.2279 - accuracy: 0.1255 - val_loss: 2.1270 - val_accuracy: 0.0968 - lr: 0.0100
Epoch 6/1000
8/8 [==============================] - 2s 209ms/step - loss: 2.2309 - accuracy: 0.1373 - val_loss: 2.2128 - val_accuracy: 0.1613 - lr: 0.0100
Epoch 7/1000
8/8 [==============================] - 2s 210ms/step - l

In [ ]:
landmarks_len

258

In [49]:
FRAMES

64

In [34]:
TRIAL = "models/vague-totem-27"
model.save('{}.h5'.format(TRIAL))

In [69]:
a = np.random.uniform(-6, 1)
a

-0.055664783546946595

In [70]:
10 ** a

0.8797012649422729

In [35]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [39]:
test_ds

<PrefetchDataset shapes: ((None, None, 258), (None, 10)), types: (tf.float32, tf.float32)>

In [44]:
from sklearn.metrics import confusion_matrix

y_pred = model.predict(test_ds)

predicted_categories = tf.argmax(y_pred, axis=1)

true_categories = tf.concat([y for x, y in test_ds], axis=0)

confusion_matrix(predicted_categories, true_categories)

ValueError: Classification metrics can't handle a mix of multiclass and multilabel-indicator targets

In [45]:
y_pred = []  # store predicted labels
y_true = []  # store true labels

# iterate over the dataset
for image_batch, label_batch in test_ds:   # use dataset.unbatch() with repeat
   # append true labels
   y_true.append(label_batch)
   # compute predictions
   preds = model.predict(image_batch)
   # append predicted labels
   y_pred.append(np.argmax(preds, axis = - 1))

# convert the true and predicted labels into tensors
correct_labels = tf.concat([item for item in y_true], axis = 0)
predicted_labels = tf.concat([item for item in y_pred], axis = 0)

In [46]:
correct_labels

<tf.Tensor: shape=(33, 10), dtype=float32, numpy=
array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0.,

In [48]:
predicted_labels

<tf.Tensor: shape=(33,), dtype=int64, numpy=
array([3, 6, 8, 9, 3, 9, 2, 7, 8, 3, 5, 8, 7, 0, 0, 7, 0, 2, 0, 1, 2, 9,
       2, 4, 8, 9, 0, 7, 0, 0, 7, 8, 4], dtype=int64)>

In [52]:
to_categorical(predicted_labels)

TypeError: Dtype of argument `keys` must be <dtype: 'string'>, received: <dtype: 'int64'>